## Beam problem: Eigenvalue method

### Dependencies

In [ ]:
from meshes import *
from Eigenvalues import *
from NumericalSolutions import *
from DynamicSolutions import *

import matplotlib.pyplot as plt
import numpy as np


"""
For the widgets to show up, do the following in the anaconda prompt:

conda install -c conda-forge ipywidgets
jupyter labextension install @jupyter-widgets/jupyterlab-manager
"""
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, interactive, fixed, interact_manual

In [ ]:
# Spatial resolution of mesh
N = 50

# Right limit of mesh
L = 1

# Generate 1D mesh
[nodes, elems, faces] = get_mesh_1D(N, [0,L], True);
elems = np.array(elems)   

In [ ]:
E=1
I=1
mu_const = 1    
two_sided_support = True

In [ ]:
### ----- Problem parameters (only used for initial solution) ------ ###

M0 = 0
ML = 0
QL = 0
a0 = 0
aL = 0
a = 0
b = 0
q = 10

In [ ]:
### ----- Get initial solution as solution to static problem ----- ###
if two_sided_support:
    
    # Create dynamic solution object in order to calculate 
    # initial numercical solution, mass and stiffness matrices
    initial_conditions = {"ML":ML, "M0":M0, "a0":a0, "aL":aL, "q":q}
    sol_dyn = DynamicSolutionBothEnds(E=E, I=I, N=N, L=L, 
                      initial_conditions = initial_conditions,
                      parameters = initial_conditions)
    w_num, wp_num = sol_dyn.initial_object.solve()
    M_ext = sol_dyn.M_ext
    S_ext = sol_dyn.S_ext
    

else:
    
    # Create dynamic solution object in order to calculate 
    # initial numercical solution, mass and stiffness matrices
    initial_conditions = {"a":a, "b":b, "QL":QL, "ML":ML, "q":q}
    sol_dyn = DynamicSolutionCantilever(E=E, I=I, N=N, L=L, 
                      initial_conditions = initial_conditions,
                      parameters = initial_conditions)
    w_num, wp_num = sol_dyn.initial_object.solve()
    M_ext = sol_dyn.M_ext
    S_ext = sol_dyn.S_ext



In [ ]:
# Create time array
T = 10   # end time
Nt = 200 # time step size
times = np.linspace(0, T, Nt)

# Initial values
w_init = np.zeros(np.shape(M_ext[:-2,:-2])[0])
wp_init = np.zeros(np.shape(M_ext[:-2,:-2])[0])

# Create solution object
sol_eigen = Eigenvalues(w0=np.ravel([w_num,wp_num],'F'), wp0=wp_init, Me=M_ext, Se=S_ext)
M = sol_eigen.M

# Get transient solutions (w[:, k] should be solution at time step k etc.) 
w, mu = sol_eigen.solver(times)


In [ ]:
def sol_step(u, i):
    """ 
    Function to separate derivative and solution 
    """
    u = u[:, i]
    ux = u[::2]
    
    return ux

In [ ]:
# get animations

%matplotlib inline

from matplotlib import animation, rc
from IPython.display import HTML

fig, ax = plt.subplots(figsize=(10,10))

ax.set_xlim(( 0, 1.2))
ax.set_ylim((-1,1))
        
line, = ax.plot([], [], lw=2)

def init():
    line.set_data([], [])
    return (line,)

def animate(i):
    line.set_data(nodes, sol_step(w, i))
    return (line,)

anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=Nt, interval=10, 
                               blit=True)
HTML(anim.to_jshtml())

In [ ]:
def eigenvalues_analytical(k, two_sided_support):
    # Input: no. of eigenvalues k and problem type
    # Returns: array of k eigenvalues
    
    if two_sided_support:
        kappa_j = lambda j: (j-0.5)*np.pi/L
    else:
        kappa_j = lambda j: j*np.pi/L
        
    return [(E*I)/mu_const*kappa_j(j)**4 for j in range(1,k+1)]

In [ ]:
plt.figure(figsize=(9, 7))
rg = 2*M.shape[0] - S_ext.shape[0]
eigenvalues, eigenvectors = sol_eigen.get_eigen()

plt.plot(np.arange(1,rg+1), 1/(np.sqrt(eigenvalues)),'r-.',label=r'numerical eigenfreq. $\omega_k=1/\sqrt{\lambda_k}$')
plt.plot(np.arange(1,rg+1), 1/(eigenvalues),'g-.',label=r'numerical eigenfreq. $\omega_k=1/\lambda_k$')
plt.plot(np.arange(1,rg+1), eigenvalues_analytical(rg, two_sided_support), 'k-.', 
         label=r'analytical eigenfreq. $\omega_k = \frac{EI}{\mu}\kappa_j^4$')
plt.xlabel('k',fontsize=19)
plt.xlim(0,10)
plt.ylim(0,1e4)
plt.legend(prop={'size': 17})
plt.show()